# Timetable Scheduler for University Classes

Timetable scheduler is a `Generative-AI` based app, developed in `langchain` using OpenAI `gpt-4` model. It takes subjects and students data in CSV format and generates schedule for each individual student.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
import csv

def read_data_from_csv(filename):
    data = []

    with open(filename, 'r') as csvfile:
        return list(csv.DictReader(csvfile))

In [5]:
subjects_data = read_data_from_csv("subjects.csv")
students_data = read_data_from_csv("students.csv")

In [6]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model="gpt-4", temperature=0)

In [9]:
system_message_prompt = SystemMessagePromptTemplate.from_template(
    "You are an expert university class scheduler with proficiency in interpreting JSON."
)
human_message_prompt = HumanMessagePromptTemplate.from_template(
"""
The following subjects` data contains subject names, the total number of classes to be attended by a student, and the available class slots for each subject.
{subjects_data}

Here is the data for a specific student, including his registered subjects and considerations. Your task is to schedule classes for these subjects.
{student}

The number of classes per week for each subject must be the same as mentioned in the data. You must make sure that classes should not overlap. If there is a conflict between the student's considerations and the scheduling, ignore the consideration. Include a brief comment at the end regarding the extent to which the considerations have been met, specifying the subject names.

The timetable should be prepared in the following format:
StudentName:
Subject1:
Classes list
Subject2:
Classes list
Subject3:
Classes list
...

Remember that classes must not overlap, you can ignore considerations when needed.
"""
)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [10]:
for student in students_data:
    response = chain.run(subjects_data=subjects_data, student=student)
    print(response, '\n')

Here is the class schedule for John Doe:

StudentName: John Doe

Subject1: Mathematics
Classes list:
- Tuesday 1:00 PM - 2:30 PM
- Tuesday 3:00 PM - 4:30 PM
- Thursday 1:00 PM - 2:30 PM

Subject2: Science
Classes list:
- Tuesday 9:00 AM - 10:30 AM
- Thursday 11:00 AM - 12:30 PM
- Friday 1:00 PM - 2:30 PM

Subject3: History
Classes list:
- Wednesday 1:00 PM - 2:30 PM
- Wednesday 3:00 PM - 4:30 PM

Comment: The student's consideration to avoid classes on Mondays and Fridays has been partially met. For the subject Mathematics, all classes are scheduled on Tuesday and Thursday. For the subject History, all classes are scheduled on Wednesday. However, for the subject Science, one class had to be scheduled on Friday due to the limited availability of class slots and to avoid overlap with other subjects.

Please note that the schedule has been prepared to ensure that there are no overlapping classes. The student's considerations were taken into account to the extent possible, but the priority